# Pizza Tracker
### This short notebook will request tracking information from Domino's for an order associated with a phone number

In [ ]:
import requests 
import xmltodict
import time

# Enter the phone number for the order you'll be tracking
# please note that format does matter - it must match the one you sent to Domino's
PHONE_NUMBER = '5555555555'  

In [ ]:
# These functions will help us track the order as it goes through Domino's system.

def add_time(time, emoji):
    '''Generates a line segment with a notch for each minute.'''
    result = ''
    if time:
        result += emoji
        for i in range(int(time) // 60):
            result += '='
    return result
        
def show_status(data):
    '''Generates and prints a ASCII art timeline of our delivery's progress'''
    result = ''
    done = False
    for stuff in [('MakeTimeSecs', '📱'), 
                  ('OvenTimeSecs', '🔥'), 
                  ('RackTimeSecs', '♨️'), 
                  ('OrderDeliveryTimeSecs', '🚘')]:
        key, emoji = stuff
        result += add_time(data[key], emoji)
    if data['OrderStatus'] == 'Complete':
        result += '😊'
        done=True
    print(result)
    return done

In [ ]:
# You can test your tracking here.  This should return the data representing the status of your pizza

r = requests.get('https://trkweb.dominos.com/orderstorage/GetTrackerData', params={'Phone': PHONE_NUMBER})
x = xmltodict.parse(r.text)
data = x['soap:Envelope']['soap:Body']['GetTrackerDataResponse']['OrderStatuses']['OrderStatus']
data

In [ ]:
# When you run this cell, it will loop until the pizza is here. 
# Every 3 seconds it will print an updated tracker.

while not show_status(data):    # print our tracker and return `True` if the pizza is here.
    # if it's not here, we wait 3 seconds
    time.sleep(3)
    
    # and then update the tracking info
    r = requests.get('https://trkweb.dominos.com/orderstorage/GetTrackerData', params={'Phone': PHONE_NUMBER})
    x = xmltodict.parse(r.text)
    data = x['soap:Envelope']['soap:Body']['GetTrackerDataResponse']['OrderStatuses']['OrderStatus']
